In [1]:
# Install llama-cpp

!set LLAMA_CUBLAS=1
!set CMAKE_ARGS=-DLLAMA_CUBLAS=on
!set FORCE_CMAKE=1

!python -m pip install llama-cpp-python==0.2.7 --prefer-binary --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122

Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00


In [2]:
# Download Airavata.gguf

import os
from huggingface_hub import hf_hub_download

def read_token(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.readline().strip()
    except FileNotFoundError:
        raise ValueError(f"Token file not found: {file_path}")

# Define the model name and file
model_name = "ai4bharat/Airavata"
model_file = "Airavata.gguf"

# Download the model from Hugging Face Hub
model_path = hf_hub_download(
    model_name,
    filename=model_file,
    local_dir='models/',  # Download the model to the "models" folder
)

print("Model path:", model_path) # models/Airavata

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Airavata.gguf:   0%|          | 0.00/13.7G [00:00<?, ?B/s]

Model path: models/Airavata.gguf


In [8]:
from llama_cpp import Llama

MESSAGES = []

def create_prompt_with_chat_format(messages, bos="<s>", eos="</s>", add_bos=True):
    formatted_text = ""
    for message in messages:
        if message["role"] == "system":
            formatted_text += "<|system|>\n" + message["content"] + "\n"
        elif message["role"] == "user":
            formatted_text += "<|user|>\n" + message["content"] + "\n"
        elif message["role"] == "assistant":
            formatted_text += "<|assistant|>\n" + message["content"].strip() + eos + "\n"
        else:
            raise ValueError(
                "Tulu chat template only supports 'system', 'user' and 'assistant' roles. Invalid role: {}.".format(
                    message["role"]
                )
            )
    formatted_text += "<|assistant|>\n"
    formatted_text = bos + formatted_text if add_bos else formatted_text
    return formatted_text


def select_llm() -> Llama:
    return Llama(model_path="models/Airavata.gguf", n_gpu_layers=-1, n_threads=2, n_ctx=4096, verbose=True)

def get_answer(llm, messages, memory=5) -> str:
    generation_kwargs = {
      "max_tokens":200,
      "stop":["</s>"],
      "echo":False,
      "top_k":50,
      "top_p":0.5
    }
    res = llm(create_prompt_with_chat_format(messages[-5:], add_bos=False), **generation_kwargs)

    return res["choices"][0]["text"]

In [4]:
llm = select_llm()

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
from IPython.display import display, HTML

from airavata_html import chat_html, js_code, update_html

MESSAGES = []

# Display the HTML and JavaScript
display(HTML(chat_html + js_code))


from IPython.display import display, HTML

# Define the Python function to process the input
def process_input(user_input):
    global MESSAGES, update_html
    print("User:",user_input, "\n")
    MESSAGES.append({"role": "user", "content": user_input})
    response = get_answer(llm, MESSAGES)
    MESSAGES.append({"role": "assistant", "content": response})
    print("Airavata:",response,"\n")

    user_input_escaped = user_input.replace("'", "\\'").replace('"', '\\"')
    response_escaped =  response = response.replace('\n', '<br>').replace("'", "\\'").replace('"', '\\"')


    display(HTML(update_html.format(user_input=user_input_escaped, response=response_escaped)))

# Register the Python function with the notebook
from google.colab import output
output.register_callback('process_input', process_input)
print("[Debug]\n")

[Debug]

User: Hi 



Llama.generate: prefix-match hit


Airavata: Hello! How can I help you today? 



User: How many types of passports are there? 



Llama.generate: prefix-match hit


...

Llama.generate: prefix-match hit


Airavata: There are several different types of visas that are issued by various countries for different purposes, such as traveling, studying, working, or living in a particular country. Some common types include:

1. Tourist Visa: This is the most commonly used type of visa and is issued to foreigners who want to visit a particular country for tourism purposes. It typically has a validity period of 3 months to one year, depending on the country.
2. Student Visa: These are issued to foreign students who need to obtain permission from their home country to study in another country. They usually have specific requirements such as proof of enrollment and financial support.
3. Work Visa: These are issued to foreigners who want to work in a particular country for long periods of time, typically up to 5 years or more. They require certain qualifications and certifications, as well as proof of employment and income.
4. Resident 



User: In general, what type of questions have I been asking? 



Llama.generate: prefix-match hit


Airavata: You've asked a variety of questions about different types of passports and visas, including the number of each type, their requirements, and how they 

